🧠 What we’ll build

A local Research Assistant Agent that can:

🔹 Summarize a piece of text<br>
🔹 Extract key topics<br>
🔹 Perform a simple sentiment analysis<br>
🔹 Return a structured, reasoned summary<br>

In [26]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
import re

Step 1. Connect to local model

In [27]:
llm = Ollama(model="mistral")  # or "llama3", "phi3", etc.

Step 2. Define tools (functions agent can use)

In [28]:
@tool
def summarize_text(text: str) -> str:
    """Summarizes the given text in 3–4 lines."""
    sentences = re.split(r'(?<=[.!?]) +', text)
    summary = " ".join(sentences[:3])
    return f"Summary: {summary}"

@tool
def extract_topics(text: str) -> str:
    """Extracts key topics or keywords from text."""
    words = re.findall(r'\b[A-Z][a-z]+\b', text)
    unique = sorted(set(words))
    return f"Topics: {', '.join(unique[:5])}"

@tool
def sentiment_analysis(text: str) -> str:
    """Performs a very simple sentiment check."""
    positive = len(re.findall(r"\b(good|great|happy|love|excellent|positive)\b", text.lower()))
    negative = len(re.findall(r"\b(bad|sad|angry|poor|hate|negative)\b", text.lower()))
    if positive > negative:
        return "Sentiment: Positive"
    elif negative > positive:
        return "Sentiment: Negative"
    else:
        return "Sentiment: Neutral"

In [29]:
# Add all tools to the toolbox
tools = [summarize_text, extract_topics, sentiment_analysis]


Step 3. Create Agent

In [30]:
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

/tmp/ipykernel_36347/943014321.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


Step 4. Run a complex query

In [22]:
text_input = """
OpenAI has made a huge impact on the world of artificial intelligence. 
Their models like ChatGPT and GPT-4 are being used in education, marketing, 
healthcare, and many other industries. People are excited but also concerned 
about job automation and ethical AI usage. The technology is powerful, 
but it must be handled responsibly.
"""

In [31]:
query = f"Please analyze this text:\n{text_input}\nPerform summary, topic extraction, and sentiment analysis."

In [32]:
result = agent.run(query)
print(result)

/tmp/ipykernel_36347/3391044460.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)




> Entering new AgentExecutor chain...
 First, I will summarize the text to get a brief overview of its content. Then, I will extract key topics mentioned in the text. Lastly, I will perform sentiment analysis to understand the general tone of the text.

Action: summarize_text
Action Input: OpenAI has made a huge impact on the world of artificial intelligence. Their models like ChatGPT and GPT-4 are being used in education, marketing, healthcare, and many other industries. People are excited but also concerned about job automation and ethical AI usage. The technology is powerful, but it must be handled responsibly.
Observation: Summary: OpenAI has made a huge impact on the world of artificial intelligence. Their models like ChatGPT and GPT-4 are being used in education, marketing, healthcare, and many other industries. People are excited but also concerned about job automation and ethical AI usage.
Thought: Now that I have summarized the text, I will extract key topics mentioned in it